In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import keras

In [ ]:
!kaggle datasets download -d matheuseduardo/flickr-faces-dataset-resized

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:


import opendatasets as od

od.download("https://www.kaggle.com/datasets/matheuseduardo/flickr-faces-dataset-resized")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: davilearn
Your Kaggle Key: ··········


100%|██████████| 7.93G/7.93G [02:53<00:00, 49.1MB/s]


In [7]:
dataset = keras.preprocessing.image.image_dataset_from_directory(
    "/content/flickr-faces-dataset-resized/64x64", label_mode=None, image_size=(64, 64), batch_size=16
)
dataset = dataset.map(lambda x: x / 255.0)

Found 52001 files belonging to 1 classes.


In [16]:
tamanho_do_ruido = 128

Descriminador = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="Descriminador",
)
Descriminador.summary()




Gerador = keras.Sequential(
    [
        keras.Input(shape=(tamanho_do_ruido,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"),
    ],
    name="Gerador",
)
print(Gerador.summary())






Model: "Descriminador"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 32, 32, 64)        3136      
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 8, 8, 128)         262272    
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 256)       

In [17]:

class GAN(keras.Model):
    def __init__(self, Descriminador, Gerador, tamanho_do_ruido=128):
        super(GAN, self).__init__()#roda o init d keras.model
        self.Descriminador = Descriminador
        self.Gerador = Gerador
        self.tamanho_do_ruido = tamanho_do_ruido

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        
        batch_size = tf.shape(real_images)[0]
        #                     quantas imgs tem na base d dados
        ruido = tf.random.normal(shape=(batch_size, self.tamanho_do_ruido))


        imgs_geradas = self.Gerador(ruido)


        imgs_combinadas = tf.concat([imgs_geradas, real_images], axis=0)

        #imgs falsas = 1    imgs reais = 0
        classes = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        
        #por algum motivo do alem se vc add ruido na classe os resultados ficam melhores então...
        classes += 0.05 * tf.random.uniform(tf.shape(classes))

        # treina o descriminador
        with tf.GradientTape() as tape:
            predictions = self.Descriminador(imgs_combinadas)
            d_loss = self.loss_fn(classes, predictions)
        grads = tape.gradient(d_loss, self.Descriminador.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.Descriminador.trainable_weights)
        )

        # mais ruido
        mais_ruido = tf.random.normal(shape=(batch_size, self.tamanho_do_ruido))

        # Assemble classes that say "all real images"
        misleading_classes = tf.zeros((batch_size, 1))

        # Train the Gerador (note that we should *not* update the weights
        # of the Descriminador)!
        with tf.GradientTape() as tape:
            predictions = self.Descriminador(self.Gerador(mais_ruido))
            g_loss = self.loss_fn(misleading_classes, predictions)
        grads = tape.gradient(g_loss, self.Gerador.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.Gerador.trainable_weights))

        # atualiza as metricas
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }

In [22]:
!mkdir generated_img_
class Callback_q_monitora_a_gan(keras.callbacks.Callback):
    def __init__(self, num_img=2, tamanho_do_ruido=128):
        self.num_img = num_img
        self.tamanho_do_ruido = tamanho_do_ruido

    def on_epoch_end(self, epoch, logs=None):
        # with open('Gerador.pickle', 'wb') as f:
        #     pickle.dump(self.model.Gerador, f)
        # with open('Descriminador.pickle', 'rb') as f:
        #     self.model.Descriminador = pickle.load(f)
            
        #salva com keras
        self.model.Descriminador.save('Descriminador_keras.h5')
        self.model.Gerador.save('Gerador_keras.h5')
        
        ruido = tf.random.normal(shape=(self.num_img, self.tamanho_do_ruido))
        imgs_geradas = self.model.Gerador(ruido)
        imgs_geradas *= 255
        imgs_geradas.numpy()
        for i in range(self.num_img):
            img = keras.preprocessing.image.array_to_img(imgs_geradas[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))

mkdir: cannot create directory ‘generated_img_’: File exists


In [ ]:

epochs = 100 

gan = GAN(Descriminador=Descriminador, Gerador=Gerador, tamanho_do_ruido=tamanho_do_ruido)
gan.compile(
    d_optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.adam_v2.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs, callbacks=[Callback_q_monitora_a_gan(num_img=10, tamanho_do_ruido=tamanho_do_ruido)]
)

#salva a classe 
with open('gan.pickle', 'wb') as f:
    pickle.dump(gan, f)


Epoch 1/100
3250/3251 [============================>.] - ETA: 0s - d_loss: 0.5652 - g_loss: 1.4830

3251/3251 [==============================] - 451s 138ms/step - d_loss: 0.5651 - g_loss: 1.4827
Epoch 2/100
3250/3251 [============================>.] - ETA: 0s - d_loss: 0.5971 - g_loss: 1.2723

3251/3251 [==============================] - 450s 138ms/step - d_loss: 0.5973 - g_loss: 1.2721
Epoch 3/100
3250/3251 [============================>.] - ETA: 0s - d_loss: 0.6140 - g_loss: 1.2930

3251/3251 [==============================] - 450s 139ms/step - d_loss: 0.6140 - g_loss: 1.2928
Epoch 4/100
3250/3251 [============================>.] - ETA: 0s - d_loss: 0.6316 - g_loss: 1.1245

3251/3251 [==============================] - 450s 138ms/step - d_loss: 0.6317 - g_loss: 1.1244
Epoch 5/100
 949/3251 [=======>......................] - ETA: 5:18 - d_loss: 0.6418 - g_loss: 1.0025